In [ ]:
%run ANNOTATIONS.ipynb

In [ ]:
%run PATTERN_OBSERVER.ipynb

In [ ]:
from ipywidgets import Text,Output,GridspecLayout # for SINGLE_TAB
from ipywidgets import Tab,Layout # for TAB_COLLECTION

In [ ]:
class SINGLE_TAB:
    def __init__(self,
        name='SINGLE_TAB',
        description='SINGLE_TAB_DESCRIPTION',
        cols=3,
        rows=4,
        ):
        self._cols=cols
        self._rows=rows
        self._times_computed=0
        
        self.name=name
        self.description=description
        self.widget=GridspecLayout(n_rows=self._rows,n_columns=self._cols)
        
        # instantiate
        for r in range(self._rows):
            for c in range(self._cols):
                self.widget[r,c]=Output()

        # draw/redraw all plots in grid
        self._update()

    '''update all plots in grid'''
    def _update(self) -> None :
        self._times_computed+=1
        
        for r in range(self._rows):
            for c in range(self._cols):
                self.widget[r,c].clear_output()
                with self.widget[r,c]:
                    display(Text('row'+str(r)+' vs '+'col'+str(c)+' = '+str(self._times_computed)))
                
SINGLE_TAB().widget

In [ ]:
'''
# example
a=SINGLE_TAB()
a.widget

a._update()
'''
None

In [ ]:
class TAB_COLLECTION(IObserver):
    def __init__(self,
        name='TAB_COLLECTION',
        description='TAB_COLLECTION_DESCRIPTION',
        tabs=None
        ,
        ):
        # defaults
        if tabs==None:
            tabs=[
                SINGLE_TAB('tab1'),
                SINGLE_TAB('tab2'),
                SINGLE_TAB('tab3'),
                SINGLE_TAB('tab4'),
            ]
        self._tabs=tabs # contents of tab collection
        self._tab_names=[x.name for x in self._tabs]
        self._is_up_to_date=[True]*len(self._tabs) # nothing selected by default, all tabs up to date by default
        
        self.name=name
        self.description=description
        self.widget=self._make()
        
    ####################################
    # observer pattern
    ####################################
    # observer
    def react(self,
        subject_name : str,
        subject_info : object
        ) -> None :
        print('OBSERVER PATTERN',':',self.name,'REACTS','subject_name',subject_name)
        print('OBSERVER PATTERN',':',self.name,'REACTS','subject_info',subject_info)
        
        if subject_name=='DATA_CONTAINER':
            print(1)
            #self._refresh_options(subject_info['all_backtests'])

    ####################################
    # buttons
    ####################################
    '''change active tab, update as needed'''
    def _onchange_tab(self,change) -> None :
        print(self.name,':','CLICK','index',self._active_tab_index,'name',self._active_tab.name)
        
        # handle delayed update
        if not self._is_up_to_date[self._active_tab_index]:
            print(self.name,':','DELAYED UPDATE','index',self._active_tab_index,'name',self._active_tab.name)
            self._active_tab._update() # update this tab
            self._is_up_to_date[self._active_tab_index]=True # flag tab as up to date
            
        # logging
        self._logging()
        
    ####################################
    # GUI
    ####################################
    '''build tab collection'''
    def _make(self) -> None :
        # make tab collection
        out=Tab(
            children=[x.widget for x in self._tabs],
            layout=Layout(width='100%',height='auto',min_height='500px')
        )

        # name tabs
        for i,tab_name in enumerate(self._tab_names):
            out.set_title(index=i,title=tab_name)
            
        # observe active tab
        out.observe(self._onchange_tab,names='selected_index')

        # return it
        return out

    '''delayed update of tab collection, update active tab immediate, delay update of non-active tabs till when they are viewed'''
    def _update(self) -> None:
        # delayed update
        for i,tab in enumerate(self._tabs):
            if self._active_tab_index==i:
                print(self.name,':','UPDATE','index',self._active_tab_index,'name',self._active_tab.name)
                tab._update() # update this particular tab
                self._is_up_to_date[i]=True # flag is up to do date
            else:
                self._is_up_to_date[i]=False # flag these tabs as out of date, update when viewed
            
        # logging
        self._logging()
    
    @property
    def _active_tab_index(self) -> int :
        return self.widget.selected_index
    
    @property
    def _active_tab(self) -> str :
        return self._tabs[self._active_tab_index]
    
    @property
    def _times_computed(self) -> List[int]:
        return [x._times_computed for x in self._tabs]
    
    def _logging(self) -> None:
        # delayed updating
        print(self.name,':','compute times',self._times_computed)
        print(self.name,':','is up to date',self._is_up_to_date)

# example
TAB_COLLECTION().widget

In [ ]:
'''
# example
a=TAB_COLLECTION(tabs=
    [
        SINGLE_TAB('a'),
        SINGLE_TAB('b'),
        SINGLE_TAB('c'),
        SINGLE_TAB('d'),
    ]
)
a.widget

# global delayed update
a._update()

# request immediate update of specific tab
a.tabs[0].update()
a.tabs[0].update()
a.tabs[2].update()
a.tabs[2].update()
a.tabs[3].update()
'''
None